### Introduction
In the past decade, both the NBA and EuroLeague have seen a notable increase in 3-point shooting. While it’s often assumed that shooting more 3s could lead to a drop in accuracy, the data may tell a different story. This project investigates whether teams have become not only more reliant on 3-pointers, but also more efficient, challenging assumptions about volume vs. efficiency in modern basketball.

### Objectives
- Analyze YoY trends in average 3PA and 3P%
- Evaluate the correlation between increased volume and accuracy
- Compare the league-wide shooting evolution in both competitions

### Hypothesis
Over the last 10 years, the rise in average 3-point attempts (av3PA) has not lowered 3-point percentage (3P%) in the NBA or EuroLeague, in fact, 3P% has improved year-over-year.

### Metrics involved
- av3P (Average 3 Point Attempts): The number of shots a team takes from beyond the three-point line per season on average.
- 3P% (Three-Point Percentage): Measures how accurately teams shoot from beyond the three point line.

In [27]:
# install the NBA API

!pip install nba_api

In [28]:
# install the Euroleague API

!pip install euroleague-api

In [546]:
# import numpy, pandas and requests

import pandas as pd
import numpy as np
import requests

## EUROLEAGUE DATA

In [311]:
######### Import euroleague data

from euroleague_api.team_stats import TeamStats


In [313]:
ts = TeamStats("E")

In [323]:
# test to see how the function works for a single year
df_euro1 = ts.get_team_stats_single_season("traditional", 2020)
df_euro1

,teamRanking,gamesPlayed,minutesPlayed,pointsScored,twoPointersMade,twoPointersAttempted,twoPointersPercentage,threePointersMade,threePointersAttempted,threePointersPercentage,...,turnovers,blocks,blocksAgainst,foulsCommited,foulsDrawn,pir,team.code,team.tvCodes,team.name,team.imageUrl
0,1,34.0,40.294118,74.1,17.3,35.2,49%,8.6,22.7,37.8%,...,13.3,1.8,2.7,21.6,20.0,74.9,RED,CZV,Crvena Zvezda Meridianbet Belgrade,https://media-cdn.incrowdsports.com/d2eef4a8-6...
1,2,34.0,40.294118,77.5,18.2,35.4,51.5%,9.3,26.8,34.6%,...,13.9,3.3,2.4,22.1,18.5,79.4,KHI,KHI,BC Khimki,https://media-cdn.incrowdsports.com/04bd9ac1-e...
2,3,34.0,40.294118,76.7,19.1,36.5,52.2%,8.4,21.4,39.3%,...,14.5,3.2,2.0,21.5,19.9,81.4,ASV,ASV,LDLC ASVEL Villeurbanne,https://media-cdn.incrowdsports.com/e33c6d1a-9...
3,4,34.0,40.588235,78.4,19.6,37.3,52.4%,8.8,25.1,35.2%,...,13.6,2.6,3.1,19.4,19.6,84.0,TEL,MTA,Maccabi Playtika Tel Aviv,https://media-cdn.incrowdsports.com/5c55ef14-2...
4,5,34.0,40.441176,78.6,19.2,34.2,56.1%,8.8,25.2,34.8%,...,14.2,2.8,2.3,20.6,20.0,85.3,OLY,OLY,Olympiacos Piraeus,https://media-cdn.incrowdsports.com/789423ac-3...
5,6,39.0,40.384615,78.6,21.3,41.7,51.1%,7.2,18.9,38.2%,...,12.6,2.2,1.8,22.8,20.5,85.5,MUN,BAY,FC Bayern Munich,https://media-cdn.incrowdsports.com/817b0e58-d...
6,7,39.0,40.128205,77.5,18.1,32.2,56.2%,9.2,24.5,37.5%,...,12.3,2.0,2.3,21.0,20.6,85.8,DYR,ZEN,Zenit St Petersburg,https://media-cdn.incrowdsports.com/4c571fd6-5...
7,8,34.0,40.000000,77.4,20.1,38.5,52.2%,8.4,20.0,42.1%,...,13.4,1.1,2.6,19.4,19.4,86.1,ZAL,ZAL,Zalgiris Kaunas,https://media-cdn.incrowdsports.com/0aa09358-3...
8,9,37.0,40.135135,77.8,21.1,37.0,57%,8.1,20.6,39.6%,...,14.2,1.9,2.3,18.9,18.5,86.6,ULK,FNB,Fenerbahce Beko Istanbul,https://media-cdn.incrowdsports.com/0233ebbb-f...
9,10,34.0,40.294118,78.6,19.0,36.1,52.7%,10.1,27.6,36.5%,...,14.8,2.2,2.9,18.4,19.5,86.8,BER,BER,ALBA Berlin,https://media-cdn.incrowdsports.com/ccc34858-2...


In [34]:
df_euro1.columns

Index(['teamRanking', 'gamesPlayed', 'minutesPlayed', 'pointsScored',
       'twoPointersMade', 'twoPointersAttempted', 'twoPointersPercentage',
       'threePointersMade', 'threePointersAttempted',
       'threePointersPercentage', 'freeThrowsMade', 'freeThrowsAttempted',
       'freeThrowsPercentage', 'offensiveRebounds', 'defensiveRebounds',
       'totalRebounds', 'assists', 'steals', 'turnovers', 'blocks',
       'blocksAgainst', 'foulsCommited', 'foulsDrawn', 'pir', 'team.code',
       'team.tvCodes', 'team.name', 'team.imageUrl'],
      dtype='object')

In [341]:
# Create lists of years with my desired rangea and a list where I will compile the season dataframes
years = list(range(2014, 2025))
season_dfs = []

# loop to obtain the df of single stats per team per season, formatting for the season, filtering for the columns that I need 
# and appending it to the list I created above

for year in years:
    df = ts.get_team_stats_single_season("traditional", year)
    
    df["season"] = f"{year}-{str(year + 1)}" 
    
    df_clean = df[["threePointersAttempted", "threePointersMade", "threePointersPercentage", "team.name", "season", "twoPointersAttempted", "twoPointersMade"]]
    
    season_dfs.append(df_clean)

In [343]:
# Combine all seasons into one DataFrame
df_euro_all = pd.concat(season_dfs)

# Rename columns
df_euro_all.rename(columns={
    "threePointersAttempted": "eur_3PA",
    "threePointersMade": "eur_3PM",
    "threePointersPercentage": "eur_3P%",
    "team.name": "team",
    "twoPointersAttempted": "eur_2PA",
    "twoPointersMade": "eur_2PM"
}, inplace=True)

df_euro_all

,eur_3PA,eur_3PM,eur_3P%,team,season,eur_2PA,eur_2PM
0,19.6,7.1,36.4%,Zalgiris Kaunas,2014-2015,39.8,19.0
1,16.8,6.0,36.1%,ALBA Berlin,2014-2015,41.5,21.3
2,24.2,7.7,31.7%,EA7 Emporio Armani Milan,2014-2015,39.4,20.2
3,21.6,7.5,34.8%,Anadolu Efes Istanbul,2014-2015,39.4,19.3
4,20.2,6.0,29.5%,Crvena Zvezda Meridianbet Belgrade,2014-2015,43.4,21.7
...,...,...,...,...,...,...,...
13,22.7,8.2,36.2%,AS Monaco,2024-2025,40.9,22.7
14,24.7,9.5,38.7%,FC Barcelona,2024-2025,40.7,23.4
15,24.3,9.4,38.5%,Anadolu Efes Istanbul,2024-2025,38.3,22.5
16,25.9,9.7,37.3%,Olympiacos Piraeus,2024-2025,35.4,20.5


In [345]:
# Strip the '%' symbol and convert to float
df_euro_all['eur_3P%'] = df_euro_all['eur_3P%'].str.rstrip('%').astype('float')

In [347]:
# Create a copy for filtering
df_euro_filtered = df_euro_all.copy()

# Group by season, find the avg and restart the index
euro_avg = df_euro_filtered.groupby("season")[["eur_3PA", "eur_3PM", "eur_3P%", "eur_2PA", "eur_2PM"]].mean().reset_index()
euro_avg

,season,eur_3PA,eur_3PM,eur_3P%,eur_2PA,eur_2PM
0,2014-2015,22.006250,7.775000,35.381250,39.762500,20.381250
1,2015-2016,22.650000,8.137500,35.868750,38.200000,19.881250
2,2016-2017,22.606250,8.400000,37.187500,38.962500,20.143750
3,2017-2018,22.812500,8.543750,37.556250,38.543750,20.306250
4,2018-2019,23.131250,8.525000,36.850000,38.331250,20.543750
5,2019-2020,24.750000,9.233333,37.244444,36.961111,19.566667
6,2020-2021,23.861111,9.005556,37.911111,36.333333,19.455556
7,2021-2022,24.177778,8.600000,35.572222,36.377778,19.411111
8,2022-2023,25.127778,9.077778,36.105556,35.766667,19.494444
9,2023-2024,25.327778,9.222222,36.372222,37.094444,20.233333


In [351]:
# Create new column for FG%, first I add the attempts, then I add the makes, and lastly I create the new column that I'm going to use in FG%.
euro_avg['eur_FGA'] = euro_avg['eur_3PA'] + euro_avg['eur_2PA']
euro_avg['eur_FGM'] = euro_avg['eur_3PM'] + euro_avg['eur_2PM']

euro_avg['eur_FG%'] = euro_avg['eur_FGM'] / euro_avg['eur_FGA'] * 100

In [365]:
euro_avg

,season,eur_3PA,eur_3PM,eur_3P%,eur_2PA,eur_2PM,eur_FGA,eur_FGM,eur_FG%,eur_3PA_ratio
0,2014-2015,22.006250,7.775000,35.381250,39.762500,20.381250,61.768750,28.156250,45.583325,35.626834
1,2015-2016,22.650000,8.137500,35.868750,38.200000,19.881250,60.850000,28.018750,46.045604,37.222679
2,2016-2017,22.606250,8.400000,37.187500,38.962500,20.143750,61.568750,28.543750,46.360776,36.717085
3,2017-2018,22.812500,8.543750,37.556250,38.543750,20.306250,61.356250,28.850000,47.020475,37.180401
4,2018-2019,23.131250,8.525000,36.850000,38.331250,20.543750,61.462500,29.068750,47.295099,37.634737
5,2019-2020,24.750000,9.233333,37.244444,36.961111,19.566667,61.711111,28.800000,46.669067,40.106230
6,2020-2021,23.861111,9.005556,37.911111,36.333333,19.455556,60.194444,28.461111,47.281957,39.640055
7,2021-2022,24.177778,8.600000,35.572222,36.377778,19.411111,60.555556,28.011111,46.256881,39.926606
8,2022-2023,25.127778,9.077778,36.105556,35.766667,19.494444,60.894444,28.572222,46.920901,41.264483
9,2023-2024,25.327778,9.222222,36.372222,37.094444,20.233333,62.422222,29.455556,47.187611,40.574938


In [357]:
# I need to create a new column that adds 2PA and 3PA, and name it as eur_FGA.
# I need to add a new column to each DataFrame that shows the proportion of 3-point attempts (3PA) out of total field goal attempts (FGA).
# I call it 3PA_ratio

In [359]:
euro_avg['eur_FGA'] = euro_avg['eur_3PA'] + euro_avg['eur_2PA']

In [363]:
euro_avg['eur_3PA_ratio'] = euro_avg['eur_3PA'] / euro_avg['eur_FGA'] *100

In [377]:
# Finally drop the columns that I'm not going to use and round the numbers

euro_avg.drop(columns=['eur_3PM', 'eur_2PM'], inplace=True)

In [381]:
euro_avg = euro_avg.round(2)

In [383]:
euro_avg

,season,eur_3PA,eur_3P%,eur_FG%,eur_3PA_ratio
0,2014-2015,22.01,35.38,45.58,35.63
1,2015-2016,22.65,35.87,46.05,37.22
2,2016-2017,22.61,37.19,46.36,36.72
3,2017-2018,22.81,37.56,47.02,37.18
4,2018-2019,23.13,36.85,47.30,37.63
5,2019-2020,24.75,37.24,46.67,40.11
6,2020-2021,23.86,37.91,47.28,39.64
7,2021-2022,24.18,35.57,46.26,39.93
8,2022-2023,25.13,36.11,46.92,41.26
9,2023-2024,25.33,36.37,47.19,40.57


## NBA DATA

In [40]:
########### import nba data

from nba_api.stats.endpoints import leaguedashteamstats

In [41]:
nba_ts = leaguedashteamstats

In [42]:
# test to see how the function works for a single year
df_nba1 = nba_ts.LeagueDashTeamStats(
        season="2015-16",
        measure_type_detailed_defense="Base",
        per_mode_detailed="PerGame")

df_nba_data = df_nba1.get_data_frames()[0]
df_nba_data.head

<bound method NDFrame.head of        TEAM_ID               TEAM_NAME  GP   W   L  W_PCT   MIN   FGM   FGA  \
0   1610612737           Atlanta Hawks  82  48  34  0.585  48.4  38.6  84.4   
1   1610612738          Boston Celtics  82  48  34  0.585  48.2  39.2  89.2   
2   1610612751           Brooklyn Nets  82  21  61  0.256  48.2  38.2  84.4   
3   1610612766       Charlotte Hornets  82  48  34  0.585  48.4  37.0  84.4   
4   1610612741           Chicago Bulls  82  42  40  0.512  48.5  38.6  87.4   
5   1610612739     Cleveland Cavaliers  82  57  25  0.695  48.4  38.7  84.0   
6   1610612742        Dallas Mavericks  82  42  40  0.512  48.8  37.4  84.1   
7   1610612743          Denver Nuggets  82  33  49  0.402  48.4  37.7  85.4   
8   1610612765         Detroit Pistons  82  44  38  0.537  48.5  37.9  86.4   
9   1610612744   Golden State Warriors  82  73   9  0.890  48.5  42.5  87.3   
10  1610612745         Houston Rockets  82  41  41  0.500  48.4  37.7  83.5   
11  1610612754        

In [43]:
df_nba_data.columns

Index(['TEAM_ID', 'TEAM_NAME', 'GP', 'W', 'L', 'W_PCT', 'MIN', 'FGM', 'FGA',
       'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'OREB',
       'DREB', 'REB', 'AST', 'TOV', 'STL', 'BLK', 'BLKA', 'PF', 'PFD', 'PTS',
       'PLUS_MINUS', 'GP_RANK', 'W_RANK', 'L_RANK', 'W_PCT_RANK', 'MIN_RANK',
       'FGM_RANK', 'FGA_RANK', 'FG_PCT_RANK', 'FG3M_RANK', 'FG3A_RANK',
       'FG3_PCT_RANK', 'FTM_RANK', 'FTA_RANK', 'FT_PCT_RANK', 'OREB_RANK',
       'DREB_RANK', 'REB_RANK', 'AST_RANK', 'TOV_RANK', 'STL_RANK', 'BLK_RANK',
       'BLKA_RANK', 'PF_RANK', 'PFD_RANK', 'PTS_RANK', 'PLUS_MINUS_RANK'],
      dtype='object')

In [325]:
df_nba_data.head(5)

,TEAM_ID,TEAM_NAME,GP,W,L,W_PCT,MIN,FGM,FGA,FG_PCT,...,REB_RANK,AST_RANK,TOV_RANK,STL_RANK,BLK_RANK,BLKA_RANK,PF_RANK,PFD_RANK,PTS_RANK,PLUS_MINUS_RANK
0,1610612737,Atlanta Hawks,82,48,34,0.585,48.4,38.6,84.4,0.458,...,24,2,22,3,5,14,7,29,12,7
1,1610612738,Boston Celtics,82,48,34,0.585,48.2,39.2,89.2,0.439,...,6,6,12,2,22,24,29,10,5,8
2,1610612751,Brooklyn Nets,82,21,61,0.256,48.2,38.2,84.4,0.453,...,23,15,19,19,25,18,2,28,26,28
3,1610612766,Charlotte Hornets,82,48,34,0.585,48.4,37.0,84.4,0.439,...,14,19,1,22,12,21,3,14,11,9
4,1610612741,Chicago Bulls,82,42,40,0.512,48.5,38.6,87.4,0.441,...,3,12,13,29,8,27,5,26,21,18


In [391]:
nba_data = []

for year in range(2014, 2025):
    season = f"{year}-{str(year + 1)[-2:]}"
    
    # API call
    stats = nba_ts.LeagueDashTeamStats(
        season=season,
        measure_type_detailed_defense="Base",
        per_mode_detailed="PerGame"
    )
    
    df_nba = stats.get_data_frames()[0]
    
    # keep only relevant columns
    df_filtered = df_nba[['TEAM_NAME', 'FG3A', 'FG3_PCT', 'FGA','FG_PCT']].copy()

    # rename columns
    df_filtered.rename(columns={
    "TEAM_NAME": "team",
    "FG3A": "nba_3PA",
    "FG3_PCT": "nba_3P%",
    "FGA": "nba_FGA",
    "FG_PCT": "nba_FG%"
    }, inplace=True)

    # add a new column name "season", to match it later with the euroleague dataframe
    df_filtered['season'] = season
    
    # append it to the nba_data list
    nba_data.append(df_filtered)
    

In [393]:
nba_data

[                      team  nba_3PA  nba_3P%  nba_FGA  nba_FG%   season
 0            Atlanta Hawks     26.2    0.380     81.7    0.466  2014-15
 1           Boston Celtics     24.6    0.327     87.9    0.443  2014-15
 2            Brooklyn Nets     19.9    0.331     83.0    0.451  2014-15
 3        Charlotte Hornets     19.1    0.318     84.5    0.420  2014-15
 4            Chicago Bulls     22.3    0.353     82.9    0.442  2014-15
 5      Cleveland Cavaliers     27.5    0.367     82.2    0.458  2014-15
 6         Dallas Mavericks     25.4    0.352     85.8    0.463  2014-15
 7           Denver Nuggets     24.8    0.325     87.3    0.433  2014-15
 8          Detroit Pistons     24.9    0.344     85.8    0.432  2014-15
 9    Golden State Warriors     27.0    0.398     87.0    0.478  2014-15
 10         Houston Rockets     32.7    0.348     83.3    0.444  2014-15
 11          Indiana Pacers     21.2    0.352     83.2    0.439  2014-15
 12    Los Angeles Clippers     26.9    0.376     8

In [395]:
df_nba_all = pd.concat(nba_data)
df_nba_all

,team,nba_3PA,nba_3P%,nba_FGA,nba_FG%,season
0,Atlanta Hawks,26.2,0.380,81.7,0.466,2014-15
1,Boston Celtics,24.6,0.327,87.9,0.443,2014-15
2,Brooklyn Nets,19.9,0.331,83.0,0.451,2014-15
3,Charlotte Hornets,19.1,0.318,84.5,0.420,2014-15
4,Chicago Bulls,22.3,0.353,82.9,0.442,2014-15
...,...,...,...,...,...,...
25,Sacramento Kings,35.2,0.357,90.1,0.478,2024-25
26,San Antonio Spurs,39.6,0.357,89.8,0.465,2024-25
27,Toronto Raptors,34.0,0.348,91.0,0.458,2024-25
28,Utah Jazz,39.8,0.350,88.7,0.454,2024-25


In [397]:
df_nba_all['nba_3P%'] = df_nba_all['nba_3P%'].astype(float)
df_nba_avg = df_nba_all.groupby('season')[['nba_3PA', 'nba_3P%','nba_FGA', 'nba_FG%']].mean().reset_index()
df_nba_avg

,season,nba_3PA,nba_3P%,nba_FGA,nba_FG%
0,2014-15,22.410000,0.349100,83.563333,0.449000
1,2015-16,24.083333,0.352767,84.570000,0.452200
2,2016-17,27.003333,0.357167,85.416667,0.457200
3,2017-18,28.996667,0.361733,86.063333,0.460267
4,2018-19,32.006667,0.355500,89.213333,0.460467
5,2019-20,34.103333,0.357767,88.803333,0.459900
6,2020-21,34.636667,0.366033,88.416667,0.466067
7,2021-22,35.180000,0.353567,88.090000,0.461233
8,2022-23,34.206667,0.360133,88.316667,0.475467
9,2023-24,35.106667,0.365667,88.903333,0.474300


### COMPARING DATAFRAMES AND STANDARDIZING VALUES/COLUMNS

In [49]:
# Standardizing "season" column

In [141]:
df_nba_all

,team,nba_3PA,nba_3P%,season
0,Atlanta Hawks,26.2,38.0,2014-15
1,Boston Celtics,24.6,32.7,2014-15
2,Brooklyn Nets,19.9,33.1,2014-15
3,Charlotte Hornets,19.1,31.8,2014-15
4,Chicago Bulls,22.3,35.3,2014-15
...,...,...,...,...
25,Sacramento Kings,35.2,35.7,2024-25
26,San Antonio Spurs,39.6,35.7,2024-25
27,Toronto Raptors,34.0,34.8,2024-25
28,Utah Jazz,39.8,35.0,2024-25


In [83]:
df_nba_all['season'].dtype

dtype('O')

In [401]:
def format_season(season):
    start_year = int(season[:4])
    end_year = start_year + 1
    return f"{start_year}-{end_year}"

In [403]:
df_nba_avg['season'] = df_nba_avg['season'].apply(format_season)
df_nba_avg

,season,nba_3PA,nba_3P%,nba_FGA,nba_FG%
0,2014-2015,22.410000,0.349100,83.563333,0.449000
1,2015-2016,24.083333,0.352767,84.570000,0.452200
2,2016-2017,27.003333,0.357167,85.416667,0.457200
3,2017-2018,28.996667,0.361733,86.063333,0.460267
4,2018-2019,32.006667,0.355500,89.213333,0.460467
5,2019-2020,34.103333,0.357767,88.803333,0.459900
6,2020-2021,34.636667,0.366033,88.416667,0.466067
7,2021-2022,35.180000,0.353567,88.090000,0.461233
8,2022-2023,34.206667,0.360133,88.316667,0.475467
9,2023-2024,35.106667,0.365667,88.903333,0.474300


In [405]:
# Standardizing "nba_3P%" & "nba_FG%" columns

In [407]:
df_nba_avg['nba_3P%'] = df_nba_avg['nba_3P%'] * 100

In [409]:
df_nba_avg['nba_FG%'] = df_nba_avg['nba_FG%'] * 100

In [421]:
df_nba_avg

,season,nba_3PA,nba_3P%,nba_FG%,nba_3PA_ratio
0,2014-2015,22.41,34.91,44.90,26.82
1,2015-2016,24.08,35.28,45.22,28.48
2,2016-2017,27.00,35.72,45.72,31.61
3,2017-2018,29.00,36.17,46.03,33.69
4,2018-2019,32.01,35.55,46.05,35.88
5,2019-2020,34.10,35.78,45.99,38.40
6,2020-2021,34.64,36.60,46.61,39.17
7,2021-2022,35.18,35.36,46.12,39.94
8,2022-2023,34.21,36.01,47.55,38.73
9,2023-2024,35.11,36.57,47.43,39.49


In [389]:
# I modified the previous functions to surface FGA in the NBA and 2PA in the Euroleague.
# I need to add a new column to each DataFrame that shows the proportion of 3-point attempts (3PA) out of total field goal attempts (FGA).
# I call it 3PA_ratio

In [413]:
df_nba_avg['nba_3PA_ratio'] = df_nba_avg['nba_3PA'] / df_nba_avg["nba_FGA"] *100

In [417]:
# Finally drop the columns that I'm not going to use and round the numbers

df_nba_avg.drop(columns=['nba_FGA'], inplace=True)

In [419]:
df_nba_avg = df_nba_avg.round(2)

### Final two dataframes

In [423]:
df_nba_avg

,season,nba_3PA,nba_3P%,nba_FG%,nba_3PA_ratio
0,2014-2015,22.41,34.91,44.90,26.82
1,2015-2016,24.08,35.28,45.22,28.48
2,2016-2017,27.00,35.72,45.72,31.61
3,2017-2018,29.00,36.17,46.03,33.69
4,2018-2019,32.01,35.55,46.05,35.88
5,2019-2020,34.10,35.78,45.99,38.40
6,2020-2021,34.64,36.60,46.61,39.17
7,2021-2022,35.18,35.36,46.12,39.94
8,2022-2023,34.21,36.01,47.55,38.73
9,2023-2024,35.11,36.57,47.43,39.49


In [425]:
euro_avg

,season,eur_3PA,eur_3P%,eur_FG%,eur_3PA_ratio
0,2014-2015,22.01,35.38,45.58,35.63
1,2015-2016,22.65,35.87,46.05,37.22
2,2016-2017,22.61,37.19,46.36,36.72
3,2017-2018,22.81,37.56,47.02,37.18
4,2018-2019,23.13,36.85,47.30,37.63
5,2019-2020,24.75,37.24,46.67,40.11
6,2020-2021,23.86,37.91,47.28,39.64
7,2021-2022,24.18,35.57,46.26,39.93
8,2022-2023,25.13,36.11,46.92,41.26
9,2023-2024,25.33,36.37,47.19,40.57


In [427]:
# Merging the two dataframes

merged_df = pd.merge(df_nba_avg, euro_avg, on='season')
merged_df

,season,nba_3PA,nba_3P%,nba_FG%,nba_3PA_ratio,eur_3PA,eur_3P%,eur_FG%,eur_3PA_ratio
0,2014-2015,22.41,34.91,44.90,26.82,22.01,35.38,45.58,35.63
1,2015-2016,24.08,35.28,45.22,28.48,22.65,35.87,46.05,37.22
2,2016-2017,27.00,35.72,45.72,31.61,22.61,37.19,46.36,36.72
3,2017-2018,29.00,36.17,46.03,33.69,22.81,37.56,47.02,37.18
4,2018-2019,32.01,35.55,46.05,35.88,23.13,36.85,47.30,37.63
5,2019-2020,34.10,35.78,45.99,38.40,24.75,37.24,46.67,40.11
6,2020-2021,34.64,36.60,46.61,39.17,23.86,37.91,47.28,39.64
7,2021-2022,35.18,35.36,46.12,39.94,24.18,35.57,46.26,39.93
8,2022-2023,34.21,36.01,47.55,38.73,25.13,36.11,46.92,41.26
9,2023-2024,35.11,36.57,47.43,39.49,25.33,36.37,47.19,40.57


In [429]:
merged_df.to_csv('cleaned_dataframe_new', index=False)

In [620]:
merged_df

,season,nba_3PA,nba_3P%,nba_FG%,nba_3PA_ratio,eur_3PA,eur_3P%,eur_FG%,eur_3PA_ratio
0,2014-2015,22.41,34.91,44.90,26.82,22.01,35.38,45.58,35.63
1,2015-2016,24.08,35.28,45.22,28.48,22.65,35.87,46.05,37.22
2,2016-2017,27.00,35.72,45.72,31.61,22.61,37.19,46.36,36.72
3,2017-2018,29.00,36.17,46.03,33.69,22.81,37.56,47.02,37.18
4,2018-2019,32.01,35.55,46.05,35.88,23.13,36.85,47.30,37.63
5,2019-2020,34.10,35.78,45.99,38.40,24.75,37.24,46.67,40.11
6,2020-2021,34.64,36.60,46.61,39.17,23.86,37.91,47.28,39.64
7,2021-2022,35.18,35.36,46.12,39.94,24.18,35.57,46.26,39.93
8,2022-2023,34.21,36.01,47.55,38.73,25.13,36.11,46.92,41.26
9,2023-2024,35.11,36.57,47.43,39.49,25.33,36.37,47.19,40.57
